In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
words_to_explore = ["python", "rock", "science", "dream"]

for word in words_to_explore:
    print(f"Similar words to {word}:")
    df = pd.DataFrame(glove_wiki_vectors.most_similar(word), columns=["Word", "Similarity"])
    display(df)

Similar words to python:


,Word,Similarity
0,monty,0.688624
1,php,0.586539
2,perl,0.578441
3,cleese,0.544668
4,flipper,0.511298
5,ruby,0.506693
6,spamalot,0.505639
7,javascript,0.503057
8,reticulated,0.498338
9,monkey,0.497641


Similar words to rock:


,Word,Similarity
0,band,0.802127
1,pop,0.764768
2,punk,0.760922
3,'n',0.730704
4,bands,0.726206
5,hop,0.714745
6,album,0.710084
7,music,0.701251
8,song,0.675784
9,rap,0.670989


Similar words to science:


,Word,Similarity
0,sciences,0.807316
1,physics,0.791470
2,institute,0.766325
3,mathematics,0.760767
4,studies,0.759045
5,research,0.758529
6,biology,0.738252
7,university,0.731453
8,psychology,0.727935
9,economics,0.727067


Similar words to dream:


,Word,Similarity
0,dreams,0.880748
1,love,0.726767
2,mind,0.682242
3,quest,0.678748
4,dreamed,0.668913
5,glory,0.663972
6,imagine,0.662387
7,hope,0.659250
8,forever,0.657701
9,moment,0.654232


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [7]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [8]:
similarities = []

for word1, word2 in word_pairs:
    similarity_score = glove_wiki_vectors.similarity(word1, word2)
    similarities.append((word1, word2, similarity_score))

similarities_df = pd.DataFrame(similarities, columns=["Word 1", "Word 2", "Similarity"])
display(similarities_df)

,Word 1,Word 2,Similarity
0,coast,shore,0.700027
1,clothes,closet,0.546276
2,old,new,0.643249
3,smart,intelligent,0.755273
4,dog,cat,0.879807
5,tree,lawyer,0.076719


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [9]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [10]:
results = []

for word in test_words:
    if word in glove_wiki_vectors:
        results.append({
            'Word': word,
            'Exists in glove_wiki_vectors': True,
        })
    else:
        results.append({
            'Word': word,
            'Exists in glove_wiki_vectors': False,
        })

results_df = pd.DataFrame(results)
display(results_df)



,Word,Exists in glove_wiki_vectors
0,covididiot,False
1,fomo,False
2,frenemies,True
3,anthropause,False
4,photobomb,False
5,selfie,False
6,pxg,False
7,pacg,False
8,cct,True
9,escc,True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [11]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [12]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [13]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283244

In [14]:
glove_wiki_vectors.similarity("white", "success")

0.3518238

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [15]:
analogy("man", "heterosexual", "woman")

man : heterosexual :: woman : ?


,Analogy word,Score
0,homosexual,0.713466
1,cohabiting,0.712436
2,same-sex,0.682566
3,unmarried,0.681679
4,lesbians,0.676701
5,heterosexuals,0.643881
6,couples,0.639169
7,lesbian,0.631802
8,marriages,0.629654
9,intercourse,0.628639


In [16]:
analogy("white", "professional", "black")

white : professional :: black : ?


,Analogy word,Score
0,amateur,0.715463
1,football,0.648091
2,player,0.633992
3,youth,0.613298
4,playing,0.611952
5,soccer,0.606520
6,professionals,0.602698
7,basketball,0.600729
8,career,0.599806
9,players,0.597421


In [17]:
analogy("rich", "educated", "poor")

rich : educated :: poor : ?


,Analogy word,Score
0,pupils,0.557099
1,elderly,0.554297
2,uneducated,0.551720
3,schooling,0.544815
4,students,0.540730
5,illiterate,0.537801
6,unemployed,0.534269
7,teachers,0.525769
8,graduates,0.522819
9,indifferent,0.521687


In [18]:
analogy("young", "energetic", "old")

young : energetic :: old : ?


,Analogy word,Score
0,easygoing,0.593288
1,affable,0.577339
2,50-year,0.577191
3,feisty,0.569620
4,70-year,0.546690
5,genial,0.546484
6,sprightly,0.543776
7,unassuming,0.540341
8,frenetic,0.540052
9,35-year,0.539281


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

From 1.4, there are some worrisome biases in these embeddings trained on Wikipedia. For example, the word "man" is to "heterosexual", is similar to "woman" is to "homosexual". This is a stereotype that is harmful. Additionally, the word "white" is to "professional" is similar to "black" is to "amateur". This is another stereotype that is harmful.

An example of how using embeddings with biases could cause harm in the real world is that if we were to use these embeddings in a hiring tool, the tool might be biased against certain groups of people. For example, if the embeddings have a bias that "white" people are more likely to be "professional", then the hiring tool might be more likely to recommend white candidates for professional roles. This could lead to discrimination against people of other ethnicities.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [19]:
from sklearn.datasets import fetch_20newsgroups

In [20]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [21]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [22]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [23]:
import re

def preprocess(text):
    # Regex to remove URLs, emails, digits and line breaks
    text = re.sub(r'http\S+|www\S+', 'URL', text)
    text = re.sub(r'\S+@\S+', 'EMAIL', text)
#    text = re.sub(r'\d+', 'NUM', text)
    text = re.sub(r'[\\\/\n\r]+', ' ', text)

    doc = nlp(text)
    clean_text = []

    for token in doc:
        if not token.is_stop and not token.is_punct and not token.like_url and not token.like_email and not token.like_num:
            lemma = token.lemma_.lower().strip()
            clean_text.append(lemma)
            
    return " ".join(clean_text)

In [24]:
df["text_pp"] = df["text"].apply(preprocess)

In [25]:
display(df)

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. sence wonder help u.s.c. pro...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit har...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup as...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat a...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,hi look software call shadow far know s...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive x faq part3 modify subject patch x11r...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference fed mandate ...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,hi get gateway 4dx-33v try configure x11r5 ...


In [26]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup as...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat a...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,hi like subscribe leadership magazine wonder ...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

Preprocessing Steps with Justifications:

- **URL Removal (replaced with 'URL' token):** Removes website links that would add noise to topic modeling
- **Email Address Removal (replaced with 'EMAIL' token):** Removes personal identifiers that don't give us anything about document topics
- **Line Break & Extra Whitespace Cleanup:** Enforces consistent spacing and text processing
- **Token-Level Cleaning:** Removed stopwords and punctuation, applied lemmatization and lowercasing, and removed any remaining URLs/emails/numbers

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

After experimenting with different numbers of topics using LDA, I chose to use 5 topics. 

I found that using 5 topics gave topics that were interpretable, and meaningful with a clear theme, which suggests that this is the natural grouping for the data.

In [27]:
from sklearn.decomposition import LatentDirichletAllocation
import mglearn

In [28]:
cv = CountVectorizer()
doc_term_matrix = cv.fit_transform(df["text_pp"])

In [29]:
n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(doc_term_matrix)

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(cv.get_feature_names_out())

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
gun           say           game          email         god           
people        go            team          file          think         
email         know          play          image         people        
state         people        year          use           know          
right         come          file          program       believe       
armenian      think         player        window        say           
government    time          win           available     jesus         
turkish       tell          entry         server        thing         
jews          like          good          run           time          
law           want          season        widget        good          




In [30]:
n_topics = 8
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(doc_term_matrix)

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(cv.get_feature_names_out())

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=4,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
gun           say           file          email         
people        go            period        file          
state         know          play          image         
armenian      people        jpeg          use           
government    come          goal          window        
turkish       tell          la            program       
jews          think         pp            available     
right         time          power         server        
law           like          shot          run           
weapon        start         win           widget        


topic 4       topic 5       topic 6       topic 7       
--------      --------      --------      --------      
god           entry         game          email         
think         file          team          atheist       
people        israel        year          exist         
know          output        p

In [31]:
n_topics = 2
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(doc_term_matrix)

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(cv.get_feature_names_out())

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=2,
    n_words=10,
)

topic 0       topic 1       
--------      --------      
people        file          
think         email         
know          program       
say           use           
god           image         
year          window        
time          include       
like          available     
come          run           
good          entry         




<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

For 5 topics, here are labels that I think are logical:

- Topic 0: Politics/Conflict (gun, people, state, armenian, turkish, etc.)
- Topic 1: General discussion (say, know, go, people, etc.)
- Topic 2: Sports/Games (game, team, play, hockey, etc.)
- Topic 3: Software/Computing (email, file, image, program, etc.)
- Topic 4: Religion/Belief (god, think, believe, jesus, etc.)

In [32]:
n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(doc_term_matrix)

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(cv.get_feature_names_out())

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
gun           say           game          email         god           
people        go            team          file          think         
email         know          play          image         people        
state         people        year          use           know          
right         come          file          program       believe       
armenian      think         player        window        say           
government    time          win           available     jesus         
turkish       tell          entry         server        thing         
jews          like          good          run           time          
law           want          season        widget        good          




<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [33]:
topic_dist = lda.transform(doc_term_matrix)

In [34]:
topics = ["Politics/Conflict", "General Discussion", "Sports/games", "Software/Computing", "Religion/Belief"]

### Document 0: Topic 0 (Politics/Conflict)

The model shows that this document has the highest association with Topic 0, which is about politics and conflict. The model is right in this case.

In [35]:
print(f"Document 0: {topic_dist[0]}")
print(f"Topic: {topics[np.argmax(topic_dist[0])]}")
print(df["text"].iloc[0])

Document 0: [0.76798266 0.14778924 0.00348735 0.00351435 0.0772264 ]
Topic: Politics/Conflict
You know, I was reading 18 U.S.C. 922 and something just did not make 
sence and I was wondering if someone could help me out.

Say U.S.C. 922 :

(1) Except as provided in paragraph (2), it shall be unlawful for
any person to transfer or possess a machinegun.

 Well I got to looking in my law dictionary and I found that a "person" 
might also be an artificial entity that is created by government 
and has no rights under the federal constitution. So, what I 
don't understand is how a statute like 922 can be enforced on 
an individual. So someone tell me how my government can tell
me what I can or cannot possess. Just passing a law 
does not make it LAW. Everyone knows that laws are constitional
until it goes to court. So, has it ever gone to court, not
just your run of the mill "Ok I had it I am guilty, put me in jail"

Has anyone ever claimed that they had a right to possess and was told
by th

### Document 1: Topic 3 (Software/Computing)

The model shows that this document has the highest association with Topic 3, which is about software and computing. The model is right in this case.

In [36]:
print(f"Document 1: {topic_dist[1]}")
print(f"Topic: {topics[np.argmax(topic_dist[1])]}")
print(df["text"].iloc[1])

Document 1: [0.00276406 0.00275644 0.00275543 0.69684816 0.2948759 ]
Topic: Software/Computing



It's not a bad question: I don't have any refs that list this algorithm
either. But thinking about it a bit, it shouldn't be too hard.

1) Take three of the points and find the plane they define as well as
the circle that they lie on (you say you have this algorithm already)

2) Find the center  of this circle. The line passing through this center
perpendicular to the plane of the three points passes through the center of
the sphere.

3) Repeat with the unused point and two of the original points. This
gives you two different lines that both pass through the sphere's
origin. Their interection is the center of the sphere.

4) the radius is easy to compute, it's just the distance from the center to
any of the original points.

I'll leave the math to you, but this is a workable algorithm. :-)


An alternate method would be to take pairs of points: the plane formed
by the perpendicular bisecto

### Document 2: Topic 4 (Religion/Belief)

The model shows that this document has the highest association with Topic 4, which is about religion and belief. The model is incorrect in this case, as the document is actually about software and computing.

In [37]:
print(f"Document 2: {topic_dist[2]}")
print(f"Topic: {topics[np.argmax(topic_dist[2])]}")
print(df["text"].iloc[2])

Document 2: [0.00344083 0.23538959 0.00343073 0.30432076 0.45341809]
Topic: Religion/Belief

Actuallay I don't, but on the other hand I don't support the idea of having
one newsgroup for every aspect of graphics programming as proposed by Brian,
in his reply to my original posting.
I would suggest a looser structure more like a comp.graphics.programmer,
comp.graphics.hw_and_sw
The reason for making as few groups as possible is for the same reason you
say we shouldn't spilt up, not to get to few postings every day.
I takes to much time to browse through all postings just to find two or 
three I'm interested in.

I understand and agree when you say you want all aspects of graphics in one
meeting. I agree to some extension. I see news as a forum to exchange ideas,
help others or to be helped. I think this is difficult to achive if there
are so many different things in one meeting.

Good evening netters|-)


### Document 3: Topic 4 (Religion/Belief)

The model shows that this document has the highest association with Topic 4, which is about religion and belief.

In [38]:
print(f"Document 3: {topic_dist[3]}")
print(f"Topic: {topics[np.argmax(topic_dist[3])]}")
print(df["text"].iloc[3])

Document 3: [0.00568726 0.00579992 0.23636948 0.74637887 0.00576447]
Topic: Software/Computing
The following problem is really bugging me,
and I would appreciate any help.

I create two windows:

w1 (child to root) with event_mask = ButtonPressMask|KeyPressMask;
w2 (child to w1) with do_not_propagate_mask = ButtonPressMask|KeyPressMask;


Keypress events in w2 are discarded, but ButtonPress events fall through
to w1, with subwindow set to w2.

FYI, I'm using xnews/olvwm.

Am I doing something fundamentally wrong here?


### Document 4: Topic 0 (Politics/Conflict)

The model shows that this document has the highest association with Topic 0, which is about politics and conflict. The model is right in this case.

In [39]:
print(f"Document 4: {topic_dist[4]}")
print(f"Topic: {topics[np.argmax(topic_dist[4])]}")
print(df["text"].iloc[4])

Document 4: [0.98167556 0.004576   0.00449908 0.00469216 0.00455721]
Topic: Politics/Conflict


  This is the latest from UPI 

     Foreign Ministry spokesman Ferhat Ataman told journalists Turkey was
     closing its air space to all flights to and from Armenia and would
     prevent humanitarian aid from reaching the republic overland across
     Turkish territory.

  
   Historically even the most uncivilized of peoples have exhibited 
   signs of compassion by allowing humanitarian aid to reach civilian
   populations. Even the Nazis did this much.

   It seems as though from now on Turkey will publicly pronounce 
   themselves 'hypocrites' should they choose to continue their
   condemnation of the Serbians.



--


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Content-based filtering works by creating feature vectors that represent the characteristics of items (like genre, actors, keywords for movies). When a user interacts with items, the system builds a profile of preferred features based on those interactions, where each row/example is an item that the user has interacted with. The recommender then matches this user profile against the feature vectors of other items to find similar ones to recommend. For example, if a user watches action movies with actors Tom Cruise and Will Smith, the system would identify these features and recommend other movies sharing similar feature vectors.

2. Two negative consequences of recommender systems include:

- Filter bubbles: Users get trapped in a "bubble" of similar content that reinforces existing preferences which can limit exposure to new recommendations that are outside their comfort zone
- Privacy concerns: These systems require collecting and analyzing large amounts of personal data about users' behaviors, preferences, and interactions, which raises issues around data collection and user privacy

3. Transfer learning in NLP refers to the technique of taking a model trained on one language task and applying it to a different but related task. This involves pre-training a model on a large dataset (like Wikipedia articles) to learn general language patterns, then fine-tuning it for specific applications like sentiment analysis or translation, allowing the model to leverage existing knowledge rather than learning from scratch.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)